# I will move this to EDA later

 Just kept it at project root so that I can import from features easily


In [ ]:
import pandas as pd
import numpy as np
from src.features.feature_engineering_maxim import *
from datetime import datetime
import pytz

from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from lightgbm import LGBMRegressor


In [ ]:
train = pd.read_csv("data/train.csv")
train = get_features(train)

# Weather

In [ ]:
weather = pd.read_csv("external_data/paris_meteostat_2020_2025.csv", parse_dates=["time"])
weather.drop(columns=["snow", "wpgt", "tsun", "coco"], inplace=True)

print(weather.isna().sum())
len(set(train["id"]) - set(weather["time"]))

In [ ]:
weather_filled = fill_weather_gaps(weather)
print(weather_filled.isna().sum())
len(set(train["id"]) - set(weather_filled["time"]))

In [ ]:
weather_final = enrich_weather_features(weather_filled)
train = merge_weather(train, weather_final)
train

# Lags & rolling

In [ ]:
train = make_lagged_features(
    train,
    pollutants=["valeur_NO2","valeur_CO","valeur_O3","valeur_PM10","valeur_PM25"],
    weather_vars=["temp","rhum","pres","vpd","abs_humidity"],
    lags=(1,3,6,12,24),
    rolls=(3,6,12,24)
)


# Pollutant Imputation

In [ ]:
BASE = LGBMRegressor(
    n_estimators=500,
    learning_rate=0.03,
    num_leaves=64,
    subsample=0.9,
    colsample_bytree=0.9,
    min_child_samples=20, 
    random_state=1
)

POLLUTANTS = ["valeur_NO2","valeur_CO","valeur_O3","valeur_PM10","valeur_PM25"]
WEATHER = ["temp","rhum","pres","wind_u","wind_v","prcp","is_wet_hour",
           "rain_3h_sum","vpd","abs_humidity","solar_elev_sin","is_daylight"]
CAL = ["is_weekend","is_holiday","month_sin","month_cos","weekday_sin",
       "weekday_cos","hour_sin","hour_cos"]




train_imputed, imputer = impute_pollutants(train, pollutants=POLLUTANTS, weather=WEATHER, cal=CAL, base=BASE)